In [25]:
#% shift enter to run section
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
import numpy as np
import numpy.ma as ma
from numpy import *
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
plt.style.use('ggplot')
import xarray as xr
from glob import glob
import netCDF4
from netCDF4 import Dataset
import matplotlib.mlab as mlab
import pylab
import pyreadr
import seaborn as sns
import tqdm
import tzlocal
import sys
import os
import scipy.interpolate as sc
from scipy import signal
my_timezone = tzlocal.get_localzone().zone
# pylab for more MATLAB like environment and inline displays plots below cells
%pylab inline

# if gsw Warning shows, manually install gsw if possible - will still work without
import glidertools as gt
from cmocean import cm as cmo  # we use this for colormaps
#del()
import geostatspy.GSLIB as GSLIB 
import geostatspy.geostats as geostats 
import scipy.stats   
from skgstat import Variogram
import pykrige

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

from sklearn.linear_model import LinearRegression
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']

Populating the interactive namespace from numpy and matplotlib


C:\Users\Sean Evans\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['f', 'pylab']
`%matplotlib` prevents importing * from pylab and numpy



In [101]:
SealIDS = np.array([1,2,4,5,13,14,15,16,17,18,19,20,21,22,23,24,25,31,47,49,50,51,52,53,57,58,62,63,64,65,66,67,68,69,70,109,110,112,113]).astype(int)

In [102]:
#%% Sean function

def calc_mldepths(flr_gridded_xds, TDiff_lim=0.1, deltaT_lim=0.25, ref_dpt=10): # This is greater than!!

    """" Therm_dep - Calculate the mixed layer depth from the rate of Temp change method (Where first differential > TDiff_lim [DegC/m])
         Tmld - Calculate the mixed layer depth from absolute diff method (Where T @ surface - T @ depth > 2 [DegC])
    Args:
      var: temperature data file
      dpt: depth data

    Return:
        time series of the mixed layer depth

    Dependencies:
        numpy

    """
    import numpy as np
    num = []
    Tmld = []
    Therm_dep = []
    
    for dive in tqdm_notebook(list(range(n))):
        v = flr_gridded_xds[:,dive].values
        d = flr_gridded_xds.depths.values

        try:
            ref_dpt_ind = np.nanargmin(np.abs(d - ref_dpt)) # gets the index for the min value for depth ignoring Nan values.
            v = v[ref_dpt_ind:]
            d = d[ref_dpt_ind:]
            Tdiff_ind = np.where(diff(v)<-TDiff_lim)[0][0]
            Therm_depi = d[Tdiff_ind] 
            Therm_dep += Therm_depi,
            num += divessummary.iloc[dive,0],

        except:
            Therm_dep += np.NaN,
            num += divessummary.iloc[dive,0],
        
        try:
            ref_dpt_ind = np.nanargmin(np.abs(d - ref_dpt)) # gets the index for the min value for depth ignoring Nan values.
            rho_diff = np.abs(v[ref_dpt_ind:] - v[ref_dpt_ind])
            x = rho_diff - deltaT_lim
            x = np.squeeze(np.where(x > 0))[0]
            Tmld_ind = x + ref_dpt_ind
            Tmld += d[Tmld_ind],            

        except:
            Tmld += np.NaN,
            #print('MLD not calculated: profile ' + str(n[dive]) + '. Setting to NaN')

    return pd.DataFrame({'num':num,'Therm_dep':Therm_dep,'Tmld':Tmld})

In [103]:
for f in tqdm_notebook(list(range(len(SealIDS)))):
    
    number = SealIDS[f]
    print(number)
    divessummary=pd.read_csv('C:/Users/Sean Evans/Documents/2020/MSc/Computing/MSc/Plots & Dive Tables/Seal'+str(number)+'/divessummary.csv')
    flr_gridded_xds = xr.open_dataarray('C:/Users/Sean Evans/Documents/2020/MSc/Computing/MSc/Gridded_Temp_TimeSeries/'+str(number)+'_flr_gridded_xds.nc')
    
    bsm_seg_df = readRDS('C:/Users/Sean Evans/Documents/2020/MSc/Computing/MSc/bsm_seg_df/'+str(number)+'_bsm_seg_df.rds')
    bsm_seg_df = pandas2ri.ri2py(bsm_seg_df)
    
    flr_gridded_xds = flr_gridded_xds.rename({'Time': 'num'})
    flr_gridded_xds.coords['num'] = bsm_seg_df.num.unique()
    flr_gridded_xds=flr_gridded_xds.sel(num=divessummary.num.values)
    n = flr_gridded_xds.shape[1]
    
    mldepths_gridded = calc_mldepths(flr_gridded_xds)
    mldepths_gridded.to_csv('C:/Users/Sean Evans/Documents/2020/MSc/Computing/MSc/Plots & Dive Tables/Seal'+str(number)+'/Thermoclines1.csv')

1


C:\Users\Sean Evans\Anaconda3\lib\site-packages\rpy2-2.9.4-py3.6-win-amd64.egg\rpy2\robjects\pandas2ri.py:191: FutureWarning:

from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.



2


4


5


...

70


109


110


112


113
